### This notebook analyzes pre-trained Traceback CNN models and their precomputed NTK (see XXX)

You will need to train CNN models on the CIFAR posioned data. These are described in https://github.com/Shawn-Shan/forensics

Then, you will need to compute the trace-NTK. This is done seperately.

We do include seperate notebooks for computeing projected-trNTK and projected-pNTK. Commented out sections of this
notebook include how to calculate traceIn kernels, which we did not report on because their kernel
calculation includes label information, which for our specific study didn't compare with the remaining kernels. A seperate
notebooks computes the Embedding kernel, 

In [1]:
import sys


import numpy as np
import torch
import random

#import matplotlib.pyplot as plt


import torch
from torch import nn, optim
from torch import load
from torch.nn import functional as F
from torch import autograd

#from torchvision import datasets

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import time

import sys
from pathlib import Path

#from numba import njit

from tqdm import tqdm

import os

#import gc

#import torchntk.autograd as ezntk

#import scipy.ndimage as ndimage

#from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
#from cleverhans.torch.attacks.projected_gradient_descent import (
#    projected_gradient_descent,
#)

#from utils2 import process_Cifar10
from einops import rearrange

import torch
import torch.nn as nn
import os

from scipy.stats import kendalltau

import pickle

In [2]:
with open('/rcfs/projects/task0_pmml/traceback/forensics/results/cifar_cifar2_res.p','rb') as f:
    RES = pickle.load(f)

In [3]:
class Model(torch.nn.Module):
    def __init__(self,):
        super(Model, self).__init__()
        
        self.conv2d = torch.nn.Conv2d(3, 32, (3,3), padding=(1,1),)
        self.batch_normalization = torch.nn.BatchNorm2d(32,momentum=0.01,eps=1e-3)
        self.conv2d_1 = torch.nn.Conv2d(32, 32, (3,3), padding=(1,1))
        self.batch_normalization_1 = torch.nn.BatchNorm2d(32,momentum=0.01,eps=1e-3)
        self.max_pooling2d = torch.nn.MaxPool2d((2,2))
        
        self.conv2d_2 = torch.nn.Conv2d(32, 64, (3,3), padding=(1,1))
        self.batch_normalization_2 = torch.nn.BatchNorm2d(64,momentum=0.01,eps=1e-3)
        self.conv2d_3 = torch.nn.Conv2d(64, 64, (3,3), padding=(1,1))
        self.batch_normalization_3 = torch.nn.BatchNorm2d(64,momentum=0.01,eps=1e-3)
        self.max_pooling2d_1 = torch.nn.MaxPool2d((2,2))
        
        self.conv2d_4 = torch.nn.Conv2d(64, 128, (3,3), padding=(1,1))
        self.batch_normalization_4 = torch.nn.BatchNorm2d(128,momentum=0.01,eps=1e-3)
        self.conv2d_5 = torch.nn.Conv2d(128, 128, (3,3), padding=(1,1))
        self.batch_normalization_5 = torch.nn.BatchNorm2d(128,momentum=0.01,eps=1e-3)
        self.max_pooling2d_2 = torch.nn.MaxPool2d((2,2))

        self.flatten = torch.nn.Flatten()
        self.max_pooling1d = torch.nn.MaxPool1d((4))
        self.dropout = torch.nn.Dropout(0.2)
        
        self.dense = torch.nn.Linear(512,512,)
        self.batch_normalization_6 = torch.nn.BatchNorm1d(512,momentum=0.01,eps=1e-3)
        
        self.dense_1 = torch.nn.Linear(512,512,)
        self.batch_normalization_7 = torch.nn.BatchNorm1d(512,momentum=0.01,eps=1e-3)
        
        self.dense_2 = torch.nn.Linear(512,10,)
    
    def forward(self,x):
        x = self.conv2d(x)
        x = torch.nn.functional.relu(x)
        x = self.batch_normalization(x)
        x = self.conv2d_1(x)
        x = torch.nn.functional.relu(x)
        x = self.batch_normalization_1(x)
        x = self.max_pooling2d(x)
        
        x = self.conv2d_2(x)
        x = torch.nn.functional.relu(x)
        x = self.batch_normalization_2(x)
        x = self.conv2d_3(x)
        x = torch.nn.functional.relu(x)
        x = self.batch_normalization_3(x)
        x = self.max_pooling2d_1(x)
        
        x = self.conv2d_4(x)
        x = torch.nn.functional.relu(x)
        x = self.batch_normalization_4(x)
        x = self.conv2d_5(x)
        x = torch.nn.functional.relu(x)
        x = self.batch_normalization_5(x)
        x = self.max_pooling2d_2(x)
        
        
        x = self.flatten(x)
        x = self.max_pooling1d(x)
        x = self.dropout(x)
        
        x = self.dense(x)
        x = torch.nn.functional.relu(x)
        x = self.batch_normalization_6(x)
        x = self.dense_1(x)
        x = torch.nn.functional.relu(x)
        x = self.batch_normalization_7(x)
        x = self.dense_2(x)
        
        return x

In [4]:
model = Model()
model.to('cuda')
torch.manual_seed(1234)

In [5]:
def calc_acc(Y,Y_hat):
    pred = torch.argmax(Y_hat,axis=1)
    truth = torch.argmax(Y,axis=1)
    return (pred==truth).sum()/len(pred)


def eval_clean_and_attack(model, X_test, Y_test, injected_X_test, injected_Y_test):
    model.eval()
    with torch.no_grad():
        X_test=X_test.to('cuda')
        Y_test=Y_test.to('cuda')
        injected_X_test=injected_X_test.to('cuda')
        injected_Y_test=injected_Y_test.to('cuda')
        
        Y_hat = model(X_test)
        clean_acc = calc_acc(Y_test,Y_hat)

        Y_hat = model(injected_X_test)
        attack_acc = calc_acc(injected_Y_test,Y_hat)
    
    return clean_acc, attack_acc

def predict_clean_and_attack(model, X_test, Y_test, injected_X_test, injected_Y_test):
    model.eval()
    with torch.no_grad():
        X_test=X_test.to('cuda')
        Y_test=Y_test.to('cuda')
        injected_X_test=injected_X_test.to('cuda')
        injected_Y_test=injected_Y_test.to('cuda')
        
        Y_hat_clean = model(X_test)

        Y_hat_poisoned = model(injected_X_test)
    
    return Y_hat_clean, Y_hat_poisoned

In [6]:
RES.keys()

train_x = rearrange(torch.tensor(RES['injected_X'],dtype=torch.float32),'b h w c -> b c h w')
train_y = torch.tensor(RES['injected_Y'],dtype=torch.float32)

test_x = rearrange(torch.tensor(RES['injected_X_test'],dtype=torch.float32),'b h w c -> b c h w')
test_y = torch.tensor(RES['injected_Y_test'],dtype=torch.float32)

test_x_og = rearrange(torch.tensor(RES['X_test'],dtype=torch.float32),'b h w c -> b c h w')
test_y_og = torch.tensor(RES['Y_test'],dtype=torch.float32)



train_loader = DataLoader(TensorDataset(train_x,train_y),batch_size=128,shuffle=True)
test_loader = DataLoader(TensorDataset(test_x,test_y),batch_size=1,shuffle=False)
test_loader_og = DataLoader(TensorDataset(test_x_og,test_y_og),batch_size=1,shuffle=False)

In [7]:
model.load_state_dict(torch.load('/rcfs/projects/task0_pmml/MODELS/poisoned_CNN.pt'))

<All keys matched successfully>

In [8]:
all_X = torch.cat([train_x,test_x,test_x_og])
all_Y = torch.cat([train_y,test_y,test_y_og])

In [9]:
injected_mask_train = np.array(RES['injected'],dtype=int)

In [10]:
NN_prediction_clean, NN_prediction_poison = predict_clean_and_attack(model, test_x_og, test_y_og, test_x, test_y)
NN_prediction_clean = torch.argmax(NN_prediction_clean,axis=1).cpu().numpy()
NN_prediction_poison = torch.argmax(NN_prediction_poison,axis=1).cpu().numpy()



In [11]:
NN_activation_clean, NN_activation_poison = predict_clean_and_attack(model, test_x_og, test_y_og, test_x, test_y)
NN_activation_clean = NN_activation_clean.cpu().numpy()
NN_activation_poison = NN_activation_poison.cpu().numpy()

In [12]:
eval_clean_and_attack(model, test_x, test_y, test_x_og, test_y_og)

(tensor(0.9986, device='cuda:0'), tensor(0.8235, device='cuda:0'))

# Load NTKs

In [13]:
def normalize(pNTK0):
    pNTK = np.ones((70_000,70_000),dtype=np.float32)
    K0 = pNTK0[0:50_000,0:50_000]
    K1 = pNTK0[50_000:60_000,0:50_000]
    K2 = pNTK0[60_000::,0:50_000]
    K3 = pNTK0[50_000:60_000,50_000:60_000]
    K4 = pNTK0[60_000::,60_000::]

    pNTK[0:50_000,0:50_000] = K0 / np.sqrt(np.diag(K0))[:,None] / np.sqrt(np.diag(K0))[None,:]
    pNTK[50_000:60_000,0:50_000] = K1 / np.sqrt(np.diag(K3))[:,None] / np.sqrt(np.diag(K0))[None,:]
    pNTK[60_000::,0:50_000] = K2 / np.sqrt(np.diag(K4))[:,None] / np.sqrt(np.diag(K0))[None,:]
    
    return pNTK[0:50_000,0:50_000], pNTK[50_000:60_000,0:50_000], pNTK[60_000::,0:50_000]

def extract(pNTK):
    return pNTK[0:50_000,0:50_000], pNTK[50_000:60_000,0:50_000], pNTK[60_000::,0:50_000]

# trNTK and trNTK^0

In [14]:
pNTK0 = np.load('/rcfs/projects/task0_pmml/traceback/kernels/NTK.npy')
K0_0, K1_0, K2_0 = extract(pNTK0)
K0, K1, K2 = normalize(pNTK0)

K1_0 = K1_0 / K0_0.std()
K2_0 = K2_0 / K0_0.std()
K0_0 = K0_0 / K0_0.std()

# Em

In [15]:
Em = np.load('/rcfs/projects/task0_pmml/traceback/kernels/embedding.npy')
K0_Em, K1_Em, K2_Em = normalize(Em)

# CK

In [16]:
CK = np.load('/rcfs/projects/task0_pmml/traceback/kernels/CK.npy')
K0_CK, K1_CK, K2_CK = normalize(CK)

# COS-Grad

In [17]:
TraceIn = np.load('/rcfs/projects/task0_pmml/traceback/kernels/TraceIn.npy')
K0_TraceIn, K1_TraceIn, K2_TraceIn = normalize(TraceIn)

# old TRAK

In [18]:
Trak = np.load('/rcfs/projects/task0_pmml/proj_trNTK/traceback_pNTK/trak_tracebackresults_pNTK/0/grads.mmap')

with torch.no_grad():
    Trak = torch.from_numpy(Trak).cuda()
    Trak = torch.matmul(Trak,Trak.T).detach().cpu().numpy()

K0_Trak, K1_Trak, K2_Trak = extract(Trak)

# proj-trNTK

In [14]:
proj_trNTK = 0
for i in tqdm(range(10)):
    A = np.load(f'/rcfs/projects/task0_pmml/proj_trNTK/traceback_trNTK/0/grads_{i}.mmap')
    A = torch.from_numpy(A).cuda()
    proj_trNTK += torch.matmul(A,A.T).cpu()


K0_proj_trNTK, K1_proj_trNTK, K2_proj_trNTK = normalize(proj_trNTK)

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [04:12<00:00, 25.21s/it]


# proj-pNTK

In [16]:
proj_pNTK = 0
A = np.load(f'/rcfs/projects/task0_pmml/proj_trNTK/traceback_pNTK10k/0/grads.mmap')
A = torch.from_numpy(A).cuda()
proj_pNTK += torch.matmul(A,A.T).cpu() 

K0_proj_pNTK, K1_proj_pNTK, K2_proj_pNTK = normalize(proj_pNTK)

# Create GLMs

In [17]:
from sklearn.linear_model import SGDClassifier
from joblib import dump, load

In [18]:
train_labels_regression = torch.argmax(all_Y,axis=1).cpu().numpy()[0:50_000]
validation_labels1 = torch.argmax(test_y,axis=1).cpu().numpy()
validation_labels2 = torch.argmax(test_y_og,axis=1).cpu().numpy()

In [19]:
def fit_and_scale(K0,K1,K2):
    LogKernelRegression = SGDClassifier(loss='log_loss',penalty='l2',alpha=1e-2,fit_intercept=True,class_weight="balanced")
    LogKernelRegression.fit(K0,train_labels_regression)
    weights = LogKernelRegression.coef_
    intercept = LogKernelRegression.intercept_
    y1 = LogKernelRegression.predict_proba(K1)
    y2 = LogKernelRegression.predict_proba(K2)
    print('poison acc: ',(LogKernelRegression.predict(K1)==validation_labels1[0:len(y1)]).sum()/len(y1))
    print('clean acc: ',(LogKernelRegression.predict(K2)==validation_labels2[0:len(y2)]).sum()/len(y2))
    return LogKernelRegression

### pNTK

In [ ]:
#pNTK_LogKernelRegression = fit_and_scale(K0,K1,K2)
#poison acc:  0.9995
#clean acc:  0.828
pNTK_LogKernelRegression = load('/rcfs/projects/task0_pmml/traceback/GLMs/pNTK.joblib')

In [ ]:
pNTK_softmax1 = pNTK_LogKernelRegression.predict_proba(K1)
pNTK_softmax2 = pNTK_LogKernelRegression.predict_proba(K2)

### pNTK0

In [ ]:
#pNTK0_LogKernelRegression = fit_and_scale(K0_0,K1_0,K2_0)
#poison acc:  0.9999
#clean acc:  0.8322
#dump(pNTK0_LogKernelRegression,'/rcfs/projects/task0_pmml/traceback/GLMs/pNTK0.joblib')
pNTK0_LogKernelRegression = load('/rcfs/projects/task0_pmml/traceback/GLMs/pNTK0.joblib')

In [ ]:
pNTK0_softmax1 = pNTK_LogKernelRegression.predict_proba(K1_0)
pNTK0_softmax2 = pNTK_LogKernelRegression.predict_proba(K2_0)

### Em

In [ ]:
#Em_LogKernelRegression = fit_and_scale(K0_Em,K1_Em,K2_Em)
#poison acc:  0.9992
#clean acc:  0.6892
Em_LogKernelRegression = load('/rcfs/projects/task0_pmml/traceback/GLMs/Em.joblib')

In [ ]:
#dump(Em_LogKernelRegression,'/rcfs/projects/task0_pmml/traceback/GLMs/Em.joblib')

In [ ]:
Em_softmax1 = Em_LogKernelRegression.predict_proba(K1_Em)
Em_softmax2 = Em_LogKernelRegression.predict_proba(K2_Em)

### CK

In [ ]:
#CK_LogKernelRegression = fit_and_scale(K0_CK,K1_CK,K2_CK)
#poison acc:  0.1861
#clean acc:  0.7885
CK_LogKernelRegression = load('/rcfs/projects/task0_pmml/traceback/GLMs/CK.joblib')

In [ ]:
CK_softmax1 = CK_LogKernelRegression.predict_proba(K1_CK)
CK_softmax2 = CK_LogKernelRegression.predict_proba(K2_CK)

### Grad-Cos

In [ ]:
#TraceIn_LogKernelRegression  = fit_and_scale(K0_TraceIn,K1_TraceIn,K2_TraceIn)
#poison acc:  0.9282
#clean acc:  0.917
TraceIn_LogKernelRegression = load('/rcfs/projects/task0_pmml/traceback/GLMs/TraceIn.joblib')

In [ ]:
TraceIn_softmax1 = TraceIn_LogKernelRegression.predict_proba(K1_TraceIn)
TraceIn_softmax2 = TraceIn_LogKernelRegression.predict_proba(K2_TraceIn)

### proj-trNTK

In [20]:
#proj_trNTK_LogKernelRegression  = fit_and_scale(K0_proj_trNTK,K1_proj_trNTK,K2_proj_trNTK)
#poison acc:  0.9995
#clean acc:  0.8105
Trak_LogKernelRegression = load('/rcfs/projects/task0_pmml/traceback/GLMs/proj_trNTK.joblib')

poison acc:  0.9995
clean acc:  0.8105


In [21]:
#dump(proj_trNTK_LogKernelRegression,'/rcfs/projects/task0_pmml/traceback/GLMs/proj_trNTK.joblib')

['/rcfs/projects/task0_pmml/traceback/GLMs/proj_trNTK.joblib']

In [27]:
proj_trNTK_softmax1 = proj_trNTK_LogKernelRegression.predict_proba(K1_proj_trNTK)
proj_trNTK_softmax2 = proj_trNTK_LogKernelRegression.predict_proba(K2_proj_trNTK)

# proj-pNTK

In [25]:
#proj_pNTK_LogKernelRegression  = fit_and_scale(K0_proj_pNTK,K1_proj_pNTK,K2_proj_pNTK)
#poison acc:  0.9993
#clean acc:  0.827 
proj_pNTK_LogKernelRegression = load('/rcfs/projects/task0_pmml/traceback/GLMs/proj_pNTK.joblib')

poison acc:  0.9993
clean acc:  0.8277


In [26]:
#dump(proj_pNTK_LogKernelRegression,'/rcfs/projects/task0_pmml/traceback/GLMs/proj_pNTK.joblib')

['/rcfs/projects/task0_pmml/traceback/GLMs/proj_pNTK.joblib']

In [28]:
proj_pNTK_softmax1 = proj_pNTK_LogKernelRegression.predict_proba(K1_proj_pNTK)
proj_pNTK_softmax2 = proj_pNTK_LogKernelRegression.predict_proba(K2_proj_pNTK)

# Taus

In [29]:
from scipy.optimize import minimize
from scipy.special import erf
from scipy.stats import norm
from math import log
from sklearn.metrics import r2_score

def numpy_softmax(x):
    return np.exp(x) / np.sum(np.exp(x),axis=1)[:,None]

def sigmoid(p):
    return 1/(1+np.exp(-p))

def logit(p,eps=0.0):
    return np.log((p+eps)/(1-p+eps))

def softmax_numpy(x):
    return np.exp(x) / np.sum(np.exp(x))

def tau_comparison_final(X,Y):    
    
    correct_logit_mask = test_y.cpu().numpy().astype(int)
    
    
    X = np.array([X[i][correct_logit_mask[i]] for i in range(len(X))])
    Y = np.array([softmax_numpy(Y[i])[correct_logit_mask[i]] for i in range(len(Y))])    

    
    mask = np.logical_or(np.logical_or(X==1,Y==1),np.logical_or(X==0,Y==0))
    X = X[~mask]
    Y = Y[~mask]
    
    Tau = kendalltau(X,Y).correlation
    print(Tau)
    #return Tau

# trNTK

In [60]:
tau_comparison_final(pNTK_softmax1,NN_activation_poison)
tau_comparison_final(pNTK_softmax2,NN_activation_clean)
# 0.5694104408561503
# 0.6428707262583747

0.5694104408561503
0.6428707262583747


# trNTK0

In [ ]:
tau_comparison_final(pNTK0_softmax1,NN_activation_poison)
tau_comparison_final(pNTK0_softmax2,NN_activation_clean)

# proj-trNTK

In [30]:
tau_comparison_final(proj_trNTK_softmax1,NN_activation_poison)
tau_comparison_final(proj_trNTK_softmax2,NN_activation_clean)
# 0.4179433388135204
# 0.56529898011184187

0.4179433388135204
0.5652989801118418


# proj-pNTK

In [31]:
tau_comparison_final(proj_pNTK_softmax1,NN_activation_poison)
tau_comparison_final(proj_pNTK_softmax2,NN_activation_clean)
# 0.5542501421213292
# 0.6648622489151715

0.5542501421213292
0.6648622489151715


# Em

In [62]:
tau_comparison_final(Em_softmax1,NN_activation_poison)
tau_comparison_final(Em_softmax2,NN_activation_clean)
# 0.2608974275890623
# 0.42985666093975866

0.2608974275890623
0.42985666093975866


# CK

In [63]:
tau_comparison_final(CK_softmax1,NN_activation_poison)
tau_comparison_final(CK_softmax2,NN_activation_clean)
# 0.4542920695531007
# 0.5524421914982721

0.4542920695531007
0.5524421914982721


# Grad-Cos

In [64]:
tau_comparison_final(TraceIn_softmax1,NN_activation_poison)
tau_comparison_final(TraceIn_softmax2,NN_activation_clean)
# -0.19739721217139808
# 0.021420640901030314

-0.19739721217139808
0.021420640901030314


# old Trak?

In [65]:
tau_comparison_final(Trak_softmax1,NN_activation_poison)
tau_comparison_final(Trak_softmax2,NN_activation_clean)
# 0.38250685716228733
# 0.6900873039298004

0.38250685716228733
0.6900873039298004
